## Strategy Idea 2 : "Cointegration - pairs trading"

__Section 0: Setup__ Importing packages/reading in data etc.

__Section 1 : Idea__ 

- __1.1__ Strategy idea

- __1.2__ Origin of idea. Context/Reasoning for strategy to work e.g. use in financial markets?

__Section 2 : Exploration__

- __2.1__ Exploratory Data Analysis. e.g plots of price/volumes that could show strategy working, how much potential.

- __2.2__ Define some 'strategy metrics'. Metrics that can can you use to gauge if this strategy will work i.e no.price points above a certain threshold that is profitable. Metrics could show how often there is an opportunity to make a trade and how much 'value' is in an opportunity e.g. how much is there a price swing?


__Section 3 : Strategy testing__

- __3.1__ Testing strategy on previous data. 

- __3.2__ State any assumptions made by testing.

- __3.3__ Model refinements. How could strategy be optimised? Careful : is this backfitting/overfitting - what measures taken to negate this e.g. bootstrapping?

- __3.4__ Assessing strategy. P/L on data sample? ROI? variance in results? longest losing run?

__Section 4 : Practical requirements__

- __4.1__ Identify if this edge is ‘realisable’? What methods will you apply to extract this value? e.g. applying a hedge function


- __4.2__ Is it possible to quantify the potential profit from the strategy? Consideration : How long will it take to obtain this? How 'risky' is it? e.g. if something did go wrong, how much do we lose? 

- __4.3__ Strategy limitations. The factors that could prevent strategy working e.g. practical considerations e.g. reacting quick enough to market updates, volume behind a price, size of bankroll needed


__Section 5: Potential limitations__

- __5.1__ What is our 'competition' - if not quantifiable, do we suspect people are doing the same thing? 

- __5.2__ So what's our edge? Identify ways of finding this edge in future? e.g what features are there? Are they predictive? Is there a certain 'market/runner' profile?





Notes (to do)
* Use lay prices as well (currently only using back prices, but two lay bets are made per pairs trade

### Section 0 : Setup

In [1]:
# importing packages
from pathlib import Path, PurePath 

import pandas as pd
import numpy as np
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns

import itertools

import utils

/Applications/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
def payout(bp, bs, lp, ls, c = 0):
    if ls == '?':
        ls = lay_hedge_stake(bp, bs, lp, c)
    elif bs == '?':
        bs = bet_hedge_stake(lp, ls, bp, c)
    loss_side = - bs + ls * (1 - c) 
    win_side = (bp - 1) * bs * (1 - c) - (lp - 1) * ls
    return win_side, loss_side 

def lay_hedge_stake(bp, bs, lp, c):
    return (((bp - 1) * bs * (1 - c)) + bs) / (lp)

def bet_hedge_stake(lp, ls, bp, c):
    return ls * (lp - c) / (bp * (1 - c) + c)

In [3]:
# reading in data
project_dir = Path.cwd().parents[2]
data_dir = project_dir / 'data' / 'processed' / 'api' / 'advanced' / 'adv_data.csv'
df = pd.read_csv(data_dir, index_col = 0)
print(df.shape)

# defining variables
back_prices = [col for col in df.columns if 'BP' in col]
back_sizes = [col for col in df.columns if 'BS' in col]
lay_prices = [col for col in df.columns if 'LP' in col]
lay_sizes = [col for col in df.columns if 'LS' in col]

df.head()

(13073, 307)


,SelectionId,MarketId,Venue,Distance,RaceType,BSP,NoRunners,BS:T-60,BS:T-59,BS:T-58,...,LS:T+5,LS:T+6,LS:T+7,LS:T+8,LS:T+9,LS:T+10,LS:T+11,LS:T+12,LS:T+13,LS:T+14
0,11986132,1.169028,Huntingdon,20.0,Chase,8.33,9,16.43,24.51,26.57,...,10.08,11.15,5.44,7.09,14.16,19.53,3.12,3.31,0.68,0.68
1,16800725,1.169028,Huntingdon,20.0,Chase,3.68,9,15.43,25.74,57.82,...,29.87,221.22,43.23,43.10,13.53,26.15,13.60,74.30,419.52,23082.10
2,20968322,1.169028,Huntingdon,20.0,Chase,14.96,9,9.87,9.25,9.15,...,37.32,6.83,4.85,11.23,16.00,5.68,40.25,12.51,10.42,13.17
3,22023486,1.169028,Huntingdon,20.0,Chase,4.25,9,84.38,64.49,58.01,...,11.67,2.02,2.02,2.02,2.02,2.02,2.02,2.02,2.02,2.02
4,24496216,1.169028,Huntingdon,20.0,Chase,6.60,9,10.64,10.11,7.91,...,34.27,54.72,11.85,17.99,48.21,17.28,38.29,6.96,4.37,4.37


### Section 1 : Pairs trading

__1.1__ **- Idea**

Pairs trading is a statistical arbitrage strategy whereby the trader attempts to exploit the tendency of some pairs of assets to have similar price changes. For example, consider two telecoms companies of similar size and profitability who are exposed to the same market conditions. Investors/traders see this similarity, and as such their prices may react in the same way to news. If there is a quantifiable relationship where the prices have a 'spread' of X on average, if the shares deviate from this average such that one is undervalued and one is overvalued, a long-short strategy can profit from a return to the average.

[Bebbington, PA (2017)](https://discovery.ucl.ac.uk/id/eprint/1563501/) looks at pairs trading in horse racing markets. 

The following outlines Bebbington's method for analysing this strategy.

* The 'signals' are the best back or lay price available at a given timestamp.
* A z-score transformation of the log of the decimal odds is used to standardise prices. This makes the relative directional movement in different prices comparable by accounting for their respective variances. 
* Non-overlapping windows of data, for example, price observations 1-5, 6-10, 11-15, are analysed to find pairs of horses. 
* Pairs are discovered by analysing the sum of squared distances between two horses' prices throughout time. Those that move the least relative to each other are the best candidates for pairs.
* Once pairs are identified, the 'spread' between their prices (on average, or at the end of each window) is compared to a minimum size requirement for a bet to be made, $\phi$.
* The 'hedging ratio' is found using an OLS regression of the price of one of the horses on the price of the other. Since the two prices are pairs but will have different variances and absolute values, their movement relative to eachother must be considered to make the strategy 'cost neutral'. It is also used to define the stake size. 
* The final observed spread indicates which on which horse a 'back-to-lay' hedge must be made (that which is expected to be backed in) and on which a 'lay-to-back' hedge must be made (that which is expected to drift). This spread is compared to an interval [?], likely a confidence interval of past spreads or simply the interval of observed spreads. If the spread is greater than usual or smaller than usual, the bets are placed. 
* Trades occur at the end of each window of data with a 5 second delay. This is used to simulate a method where the algorithm is reacting to live data. In our analysis, we look at prices for the first 30 price points and make bets in the remaining 30 periods.
* In the paper it appears that both sides of the hedge bet are made at the same point in time.

These steps are attempted below.

__1.2__ **- Setup**

**Data**

The following example will be set up with a random race and will identify tradeable pairs (or that there are none). Three DataFrames are created: (1) the unchanged race sample DataFrame with one row per horse and data going along in columns, (2) a back prices DataFrame with one column per horse and prices going through time in rows, (3) the same for lay prices. This analysis looks at prices before the race begins.

There are 60 price data points for each horse, finishing at the begining of the race.

Variables:
* $BP_{t}^{i}$ is back price for horse i at time t.

* $LP_{t}^{i}$ is lay price for horse i at time t.

In [58]:
back_prices

,h19148725,h19381603,h19679370,h19835010,h19925203,h20303798,h21214420,h21514649,h21651261,h23523793,...,h24179936,h24179939,h24590733,h25427986,h26411922,h26817712,h26830868,h27145248,h27377022,h52106
0,3.218876,3.258097,3.332205,4.094345,4.867534,4.007333,3.258097,2.602690,2.484907,5.247024,...,2.869035,2.674149,5.247024,1.916923,3.295837,3.526361,5.480639,5.480639,3.218876,1.958685
1,3.232384,3.258097,3.332205,4.094345,4.867534,4.007333,3.258097,2.602690,2.484907,5.247024,...,2.878074,2.674149,5.250649,1.916923,3.295837,3.526361,5.480639,5.480639,3.218876,1.945910
2,3.258097,3.258097,3.332205,4.094345,4.867534,4.007333,3.258097,2.602690,2.484907,5.199657,...,2.867899,2.674149,5.298317,1.916923,3.260018,3.526361,5.480639,5.480639,3.218876,1.945910
3,3.258097,3.258097,3.332205,4.094345,4.867534,4.007333,3.258097,2.602690,2.484907,5.205324,...,2.875822,2.674149,5.298317,1.916923,3.275256,3.526361,5.480639,5.480639,3.218876,1.945910
4,3.258097,3.258097,3.332205,4.094345,4.867534,4.007333,3.258097,2.602690,2.484907,5.223648,...,2.881443,2.674149,5.289680,1.916923,3.260785,3.526361,5.480639,5.480639,3.218876,1.945910
5,3.276390,3.258097,3.332205,4.132443,4.867534,4.007333,3.257327,2.602690,2.484907,5.247024,...,2.901971,2.674149,5.291293,1.916923,3.258097,3.526361,5.480639,5.480639,3.218876,1.945910
6,3.295837,3.271089,3.332205,4.174387,4.867534,4.007333,3.258097,2.602690,2.484907,5.247024,...,2.894806,2.644755,5.278268,1.916923,3.279783,3.526361,5.480639,5.480639,3.218876,1.945910
7,3.290638,3.295837,3.332205,4.174387,4.867534,4.007333,3.244154,2.602690,2.484907,5.247024,...,2.862201,2.640485,5.248812,1.916923,3.317453,3.526361,5.480639,5.480639,3.218876,1.937302
8,3.281287,3.295837,3.332205,4.189958,4.867534,4.070052,3.218876,2.634762,2.484073,5.247024,...,2.853016,2.674149,5.190008,1.916923,3.332205,3.516904,5.511411,5.480639,3.218876,1.916923
9,3.295837,3.295837,3.332205,4.248495,4.867534,4.094345,3.218876,2.639057,2.482404,5.247024,...,2.833213,2.674149,4.783149,1.916923,3.332205,3.526361,5.557986,5.480639,3.218876,1.916923


In [57]:
['SelectionId'] + back_prices

ValueError: Unable to coerce to Series, length must be 22: given 1

In [56]:
# sample_df = df[df['MarketId'] == df['MarketId'].sample(1).item()]
sample_df = df.copy()
market_id = sample_df['MarketId'].sample(1).item()

bp_df = sample_df.loc[sample_df['MarketId'] == market_id][['SelectionId'] + back_prices].copy()
# new_cols = bp_df.columns.str.replace("[BP:T]", "").str.replace("[+]", "")
# bp_df.rename(columns = dict(zip(bp_df.columns, new_cols)), inplace = True)
# bp_t_df = bp_df.T.copy()
# bp_t_df.columns = ["h" + str(int(column)) for column in bp_t_df.iloc[0]]
# bp_t_df = bp_t_df.iloc[1:-15] # using the 60 pre-off price data points
# bp_t_df.reset_index(drop=True, inplace=True)

# lp_df = sample_df[['SelectionId'] + lay_prices].copy()
# new_cols = lp_df.columns.str.replace("[LP:T]", "").str.replace("[+]", "")
# lp_df.rename(columns = dict(zip(lp_df.columns, new_cols)), inplace = True)
# lp_t_df = lp_df.T.copy()
# lp_t_df.columns = ["h" + str(int(column)) for column in lp_t_df.iloc[0]] #rename columns to horse ids
# lp_t_df = lp_t_df.iloc[1:-15] #remove horse ids, remove inplay data
# lp_t_df.reset_index(drop=True, inplace=True)

# # bsp_df = plot_df[['BSP']].copy()
# # bsp_df['min_bp'] = bsp_df['BSP'].apply(lambda x: round(utils.back_hedge_min_bp(x, 0.05), 2))
# # bsp_df['max_lp'] = bsp_df['BSP'].apply(lambda x: round(utils.lay_hedge_max_lp(x, 0.05), 2))    

# bp_t_df.head(2)

__1.3__ **- Z-score transformation**

Bebbington standardises prices by taking the natural logarithm of each price and then standardising it with a Z-score transformation. 

Taking $P_{t}^{i} = ln(BP_{t}^{i})$, this means finding 

#### $P_{t}^{'(i)} = \frac{P_{t}^{i}-\overline{P}_{t}^{i}}{\sigma^{(i)}}$

where $\sigma^{(i)}$ is the standard deviation of the horse's price throughout the time series.

The Z-score transformation gives the relationship between an individual data point in the sample relative to that of the population mean and standard deviation. This means that variations are comparable between horses.

The following will look at the first 30 observations.

In [5]:
z_bp_df = bp_t_df.copy()
z_bp_df = z_bp_df[:30] #first 30 observations
z_bp_df = np.log(z_bp_df)

for column in z_bp_df.columns:
    mean = z_bp_df[column].mean()
    sd = np.std(z_bp_df[column], ddof = 1)
    z_bp_df[column] = z_bp_df[column].apply(lambda x: (x - mean) / sd)
    
z_bp_df.head(2)

,h19148725,h19381603,h19679370,h19835010,h19925203,h20303798,h21214420,h21514649,h21651261,h23523793,...,h24179936,h24179939,h24590733,h25427986,h26411922,h26817712,h26830868,h27145248,h27377022,h52106
0,-2.614718,-1.036493,0.828063,-1.796277,1.167655,-1.619935,1.722269,-0.283758,0.698945,-0.535298,...,0.345393,0.520298,1.349473,-1.187943,-0.935382,0.520767,-1.266467,0.812547,0.254378,1.575298
1,-2.162539,-1.036493,0.828063,-1.796277,1.167655,-1.619935,1.722269,-0.283758,0.698945,-0.535298,...,1.048479,0.520298,1.362747,-1.187943,-0.935382,0.520767,-1.266467,0.812547,0.254378,1.157002


__1.4__ **- Sum of squared distances**

Following Bebbington, to select pairs, we create a matrix (DataFrame, in this case) of the sum of squared distances between pairs of horses throughout the time series.

$\Theta _{ij} = \left\{\begin{matrix}
\sum_{M}^{t=1}(P_{t}^{'(i)} - P_{t}^{'(j)})^{2}, &i\neq j\\ 
 0, i=j& 
\end{matrix}\right.$

In [6]:
ids = [column for column in z_bp_df.columns]

matrix = z_bp_df.iloc[0:0].copy()

matrix.insert(0, "horse", np.array(ids))
matrix = matrix.set_index("horse", drop = False)
del matrix["horse"]

for column in matrix.columns:
    for row in matrix.index:
        if column == row:
            matrix.loc[row, column] = np.nan
        else: 
            matrix.loc[row, column] = ((z_bp_df[row] - z_bp_df[column]) ** 2).sum() or np.nan
            
for x in range(len(ids)):
    for y in range(x, len(ids)):
        matrix.iloc[x, y] = np.nan
        
matrix

,h19148725,h19381603,h19679370,h19835010,h19925203,h20303798,h21214420,h21514649,h21651261,h23523793,...,h24179936,h24179939,h24590733,h25427986,h26411922,h26817712,h26830868,h27145248,h27377022,h52106
horse,,,,,,,,,,,,,,,,,,,,,
h19148725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h19381603,16.357687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h19679370,101.136006,88.975391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h19835010,9.782990,24.577913,91.767158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h19925203,99.599920,89.401468,15.586834,105.615237,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h20303798,14.280886,26.152005,88.677789,2.617462,101.258902,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h21214420,102.032577,90.381173,28.840295,112.595583,16.881755,115.045001,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h21514649,65.835885,63.123970,21.625432,55.791905,39.807300,49.469796,67.992352,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
h21651261,95.160386,97.896618,10.741678,86.185168,22.890299,83.840981,33.387111,25.106582,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
horse_x = matrix.min(axis=1).idxmin()
horse_y = matrix.min().idxmin()
sss = matrix.min().min()

print(f"Pair found: horse {horse_x} and horse {horse_y} with sum of squared spreads equal to {sss}.")

Pair found: horse h20303798 and horse h19835010 with sum of squared spreads equal to 2.6174616139900637.


__1.5__ **- Regression of prices of horse Y on horse X**

With $x_{t} = \left \{P_{1}^{X} + P_{2}^{X} + , ... , P_{M}^{X} \right \}$ and  $y_{t} = \left \{P_{1}^{Y} + P_{2}^{Y} + , ... , P_{M}^{Y} \right \}$ where $M$ is the final time period in the window, we carry out the OLS regression of $y_{t}$ on $x_{t}$. The estimate of $\beta$ is the hedging ratio, giving the relative holding of $x_{t}$ for a cost-neutral hedge position.

$y_{t} = \beta x_{t} + \varepsilon_{t}$

Using this estimation and the final end of window observations at time $M$ we get the spread at the end of the window: $\varepsilon_{M} = y_{M} - \hat{\beta} x_{M}$. If $\varepsilon_{M}$ is outside of an interval of past spread values such that if the spread returns to the mean a hedge bet will be profitable, bets can be made.

In [8]:
#regression setup
reg_df = bp_t_df[[horse_y, horse_x]][:30].copy() #non-standardised prices
reg_df = np.log(reg_df) 
reg_df['const'] = 1

#regression fit and results
reg = sm.OLS(endog=reg_df[horse_y], exog=reg_df[['const', horse_x]], missing='drop')

results = reg.fit()

constant = results.params[0]
beta = results.params[1]

#estimated final period (T=30) spread
spread = reg_df[horse_y].iloc[29].item() - constant - beta * reg_df[horse_x].iloc[29].item()

print(f"Final period estimated spread (in log prices) epsilon = {spread}.")

#Positive spread: horse Y has drifted from the mean, horse X has been backed in. If mean reversion occurs horse Y will be backed in and horse X will drift. 
# -> Back-to-lay hedge Y and lay-to-back hedge X
#Negative spread: horse X has drifted from the mean, horse Y has been backed in. If mean reversion occurs horse X will be backed in and horse Y will drift. 
# -> Back-to-lay hedge X and lay-to-back hedge Y

Final period estimated spread (in log prices) epsilon = 0.008115429826252907.


__1.6.1__ **- Pairs trade**

Bebbing describes the following steps for the pairs trade:

If the final estimated residual $\varepsilon_{M}$ is:
* Less than the lower bound of the threshold: back horse Y with £1, lay horse X with £$\beta$.
* Greater than the upper bound of the threshold: lay horse Y with £1 and back horse X with £$\beta$.

Bebbington describes using a self-imposed delay of 5 seconds to emulate the delays in placing a bet on the Betfair Exchange. Since each time step in this dataset before the beginning of the race is 2 minutes, 1 step will be used. The prices at time 31 (index = 30) will be used.

Put alone, this strategy doesnt make sense. The bettor is still exposed to the result of the race without closing out each position with an opposing bet on each horse after some period of time.

__1.6.2__ **- Pairs trade 2**

In a pairs trade with two financial assets (say, company shares) the objective is to go long on the share whose price is expected to increase, while shorting the share whose price is expected to decrease. In both cases the positions would be closed out: the asset is sold or bought back (in the case of the short-sale) once there is a gain or possibly after a set interval. In the prior example, the closing out of each position should be done via an opposite back/lay.

If the final estimated residual $\varepsilon_{M}$ is:
* Less than the lower bound of the threshold: back horse Y with £1, lay horse X with £$\beta$. **After k periods, lay horse Y and back horse X with the optimal stakes given the prevailing prices, as defined in utils.py.**
* Greater than the upper bound of the threshold: lay horse Y with £1 and back horse X with £$\beta$. **After k periods, back horse Y and lay horse X with the optimal stakes.**

In [9]:
# k = 35

# if spread < 0:
#     #back to lay Y
#     bp_y = bp_t_df[horse_y].iloc[30]
#     lp_y = bp_t_df[horse_y].iloc[k]
    
#     win_side_y, loss_side_y = utils.payout(bp_y, 1, lp_y, '?')
#     print(f"Win side = {win_side_y}. Loss side = {loss_side_y}.")
    
#     #lay to back X
#     lp_x = bp_t_df[horse_x].iloc[30]
#     bp_x = bp_t_df[horse_x].iloc[k]
    
#     win_side_x, loss_side_x = utils.payout(bp_x, '?', lp_x, beta)
#     print(f"Win side = {win_side_x}. Loss side = {loss_side_x}.")
        
# else:
#     #lay to back Y
#     lp_y = bp_t_df[horse_y].iloc[30]
#     bp_y = bp_t_df[horse_y].iloc[k]
    
#     win_side_y, loss_side_y = utils.payout(bp_y, '?', lp_y, 1)
#     print(f"Win side = {win_side_y}. Loss side = {loss_side_y}.")
    
#     #back to lay X
#     bp_x = bp_t_df[horse_x].iloc[30]
#     lp_x = bp_t_df[horse_x].iloc[k]
    
#     win_side_x, loss_side_x = utils.payout(bp_x, beta, lp_x, '?')
#     print(f"Win side = {win_side_x}. Loss side = {loss_side_x}.")

I believe Bebbington's methodology has flaws. The following attempts to use a more robust method for pairs trading.

### Alternative approach to pairs trading

__2.0__ **- [Herlemont (2004)](http://docs.finance.free.fr/DOCS/Yats/cointegration-en%5B1%5D.pdf) paper**

Herlemont describes in detail the econometrics of pairs trading for financial market assets. The following partly follows his commentary with some additional clarifications and discussion relating to horse racing.

**2.1 - Testing for mean reversion**

The aim is to identify odds that move together and whose spread is mean reverting. For the purposes of horse racing pairs, mean reversion is essential. Our objective is to capture prices whose spread has (temporarily) deviated from its mean. If this can be found, bets can be made to take advantage of the possible reversion.

A stochastic process $y_{t}$ that is weakly stationary has the following properties for all $t$:

* $E[y_{t}] = \mu < \infty$
* $var(y_{t}) = \gamma_{0} < \infty$
* $cov(y_{t}, y_{t-j}) = \gamma_{j} < \infty, j = 1, 2, 3 ...$

(constant mean, constant variance, covariance between two observations depends only on the distance in time between them)

A weakly stationary $I(0)$ series:
* Fluctuates around its mean with a finite variance that does not depend upon time.
* Is mean-reverting: it has tendency to return to its mean.
* Has limited memory; the effect of a shock dies out. Autocorrelations die out (fairly) rapidly.

With two horse's odds, $A_{t}$ and $B_{t}$, we look at $y_{t} = \log \frac{A_{t}}{B_{t}} = \log A_{t} - \log B_{t}$. This is once again the spread between the prices of the two horses, defined slightly differently. We want to find a pair which has a weakly stationary spread. We are interested in the ($AR(1)$) process 

$y_{t} = c + \theta y_{t-1} + \varepsilon_{t}$,

or the log odds ratio over time. If this is weakly stationary, it would suggest a mean reverting process. 

The three previous conditions, and a stability condition that $|\theta|<1$ (that the process $y_{t}$ is not a random walk or that it follows an eratic positive-to-negative pattern) must hold.
______

A Dickey-Fuller stationarity test can be carried out on the log ratio of the prices to test whether a process is weakly stationary. If we carry out the regression:

$\Delta y_{t} = \mu + \omega y_{t-1} + \varepsilon_{t}$

where the null hypothesis that $\omega = 0$ is that the 'true' relationship is $\Delta y_{t} = \mu + \varepsilon_{t} \Leftrightarrow y_{t} = \mu + y_{t-1} + \varepsilon_{t}$, or a random walk with starting point $y_{0} = \mu$.

If we can reject the null hypothesis, the price ratio is weakly stationary and thereby mean-reverting.

A Dickey-Fuller test is required for each possible pair of horses in a race, or $\frac{n(n-1)}{2}$ regressions, where $n$ is the number of horses.

While we are interested in the stochastic process $y_{t}$, we do not need to carry out the regression of $y_{t} = c + \theta y_{t-1} + \varepsilon_{t}$ for the purpose of finding pairs. This relationship between a pair of odds itself is not important to quantify. We are only interested in the features of the process. 
____

*In the previous analysis, the test for whether two odds formed a pair was to find the pair with the smallest sum of absolute differences over time in the standardised prices. That method would allow maximum 1 pair to be found per race, and the validity of that pair would not be confirmed statisticallyather. Rather, the pair's feasibilty for a trade would be tested for afterwards based on profitability. I have more confidence in the approach in this section.*

**2.2 - Screening pairs**

Herlemont describes rules to ensure that market neutrality is more achievable in pairs trading. The idea is to pick stocks with very similar characteristics like same industry and similar market betas, with the intention of minimising asymmetric shocks to the price of one stock and not the other. For example in the case of two stocks, the share on which you are long is a business heavily dependent on oil, while the other share is not, a surge in oil prices which dampens profitability of your long share will likely see its price fall, ruining the pairs trade. In the case of shares, the simplest solution would be to pick shares in similar industries with similar market betas (or with similar idiosyncratic risks).

For horses, the external factors influencing prices (news about runners, changing weather conditions, etc.) will usually always have asymmetric effects. This may be avoidable through picking horses with similar fundamental characteristics. However, this is very complicated. My hope is that the pair finding mechanism picks horses where this is already the case, because the market reacts the same way to news for these horse pairs.

We cannot follow a beta-based approach because there are not 'market-wide fluctuations' of the same sort. However, there is the fact that the implied probability of all horses in the market book is equal to approximately 1. Therefore, you could say that for a given change in implied probability for one horse, the sum of the changes in the odds of all the remaining horses is the negative the change for the given horse:

$\Delta O_{i} = - \sum_{j = 1, j \neq i}^{N_{h}} \Delta O_{j} $

There is therefore interdependence between all prices across the market. It's possible that this will cause an endogeneity problem in regressions between separate horses, as the changes in the dependent variable necessarily impact the explanatory variable. However, the impact is likely to be very small, and will be smaller the greater the number of horses. 

*In Bebbington's analysis, he describes that betting £1 on one of the horses and £$\beta$ on the other creates a market neutral bet. This is incorrect, and it appears that he has misunderstood hedging in this context. In that analysis, $\beta = \frac{y_{t}}{x_{t}}$, and therefore he is simply considering the ratio of the prices of the horses, the same ratio considered when determining the optimal stake for two given prices in a hedge. It is correct that on a single horse this creates a market neutral bet, however neutrality in horse racing means neutral to the outcome of the race. Any bet neutral to the race outcome is definitively neutral to the market. When betting on separate horses, the bets on each horse must be made neutral separately. Additionally, the use of $\beta$ in staking is unneccesary. Consider the case where £$BS$ has been bet on horse A at price $BP$. Now, horse A is priced at $LP$. The optimal stake to bet on LP is £$LS = \frac{BS * BP}{LP}$. In the aforementioned regression, $BS = 1$, hence $\beta = \frac{y_{t} * 1}{x_{t}}$ is the optimal stake only for bets of £1, otherwise it would be $S*\beta$. More importantly, using the estimated $\beta$ to find the an approximation of the optimal stake makes no sense when you can simply find the optimal stake with the aforementioned equation.*

**2.3 - Trading rules**

Timing rules must be added. 

Herlemont's basic rule is "to open a position when the ratio of two share prices hits the 2 rolling standard deviation [difference from the 130-day rolling mean] and close it when the ratio returns to the mean."

To avoid opening a position on stocks that are deviating from the mean and are going to deviate further, Herlemont describes that "the position is not opened when the ratio breaks the two-standard-deviations limit for the first time, but rather when it crosses it to revert to the mean again."

This can be achieved with the horse odds, of course in far smaller time scales. The current dataset is in 5-minute intervals for the three hours before a race; this should likely be expanded.

Stop losses should be included and trade length should also be limited.

Rules:
1. Trade on pairs whose spread is reapproaching the mean from a deviated position
2. Stop loss at x% of the initial position
3. Don't hold open pairs trades for longer than x hours. 

It should be possible to quantify the average length of time required for a mean reversion and therefore the maximum logical time to hold open a position by looking at past data.

**2.4 - Other tests and considerations**

1. It should be ensured that the regression results of one price on another are not spurious (as with the regression in 2.5). $\beta$ could be statistically meaningless if it is, meaning that it makes no sense to use it.
2. I will also test whether $y_{t} = c + \theta y_{t-1} + \varepsilon_{t}$ is $I(1)$, or difference stationary. If we can rule this out, this gives more confidence in the 'weak-stationarity' of the spread over time.
3. I will look out for $\omega$ in the DF test that are close to 1 yet pass the DF test. They will have lots of features of a random walk, so the pairs exercise might be meaningless.
4. Structural breaks (in this case, large instantaneous jumps in the spread) may make series that are stationary on either side of the break appear non-stationary. This is hard to account for in testing. 

In [10]:
#3.1 - testing for mean reversion

#using non-standardised log price data
back_prices = np.log(bp_t_df[:30]).copy()
back_prices.head()

,h19148725,h19381603,h19679370,h19835010,h19925203,h20303798,h21214420,h21514649,h21651261,h23523793,...,h24179936,h24179939,h24590733,h25427986,h26411922,h26817712,h26830868,h27145248,h27377022,h52106
0,3.218876,3.258097,3.332205,4.094345,4.867534,4.007333,3.258097,2.60269,2.484907,5.247024,...,2.869035,2.674149,5.247024,1.916923,3.295837,3.526361,5.480639,5.480639,3.218876,1.958685
1,3.232384,3.258097,3.332205,4.094345,4.867534,4.007333,3.258097,2.60269,2.484907,5.247024,...,2.878074,2.674149,5.250649,1.916923,3.295837,3.526361,5.480639,5.480639,3.218876,1.945910
2,3.258097,3.258097,3.332205,4.094345,4.867534,4.007333,3.258097,2.60269,2.484907,5.199657,...,2.867899,2.674149,5.298317,1.916923,3.260018,3.526361,5.480639,5.480639,3.218876,1.945910
3,3.258097,3.258097,3.332205,4.094345,4.867534,4.007333,3.258097,2.60269,2.484907,5.205324,...,2.875822,2.674149,5.298317,1.916923,3.275256,3.526361,5.480639,5.480639,3.218876,1.945910
4,3.258097,3.258097,3.332205,4.094345,4.867534,4.007333,3.258097,2.60269,2.484907,5.223648,...,2.881443,2.674149,5.289680,1.916923,3.260785,3.526361,5.480639,5.480639,3.218876,1.945910


In [11]:
#create dataframe where each column is log(horse a's prices) - log(horse b's prices). one new column for all n(n-1)/2 possible pairs

#use itertools to find all possible comination pairs
combos = list(itertools.combinations(back_prices.columns, 2))

#creating dataframe
for pair in combos:
    if pair == combos[0]:
        new_series = back_prices[pair[0]] - back_prices[pair[1]]
        dickey_fuller_df = pd.DataFrame(new_series)
    else:
        new_series = back_prices[pair[0]] - back_prices[pair[1]]
        dickey_fuller_df = pd.concat([dickey_fuller_df, new_series], axis=1)
        
#naming columns
dickey_fuller_df.columns = [pair[0] + "_" + pair[1] for pair in combos]

dickey_fuller_df['const'] = 1

dickey_fuller_df.head()

,h19148725_h19381603,h19148725_h19679370,h19148725_h19835010,h19148725_h19925203,h19148725_h20303798,h19148725_h21214420,h19148725_h21514649,h19148725_h21651261,h19148725_h23523793,h19148725_h23742637,...,h26817712_h27145248,h26817712_h27377022,h26817712_h52106,h26830868_h27145248,h26830868_h27377022,h26830868_h52106,h27145248_h27377022,h27145248_h52106,h27377022_h52106,const
0,-0.039221,-0.113329,-0.875469,-1.648659,-0.788457,-0.039221,0.616186,0.733969,-2.028148,-0.693147,...,-1.954278,0.307485,1.567675,0.0,2.261763,3.521954,2.261763,3.521954,1.260190,1
1,-0.025712,-0.099820,-0.861960,-1.635150,-0.774949,-0.025712,0.629694,0.747478,-2.014640,-0.679639,...,-1.954278,0.307485,1.580450,0.0,2.261763,3.534729,2.261763,3.534729,1.272966,1
2,0.000000,-0.074108,-0.836248,-1.609438,-0.749237,0.000000,0.655407,0.773190,-1.941560,-0.653926,...,-1.954278,0.307485,1.580450,0.0,2.261763,3.534729,2.261763,3.534729,1.272966,1
3,0.000000,-0.074108,-0.836248,-1.609438,-0.749237,0.000000,0.655407,0.773190,-1.947228,-0.653926,...,-1.954278,0.307485,1.580450,0.0,2.261763,3.534729,2.261763,3.534729,1.272966,1
4,0.000000,-0.074108,-0.836248,-1.609438,-0.749237,0.000000,0.655407,0.773190,-1.965551,-0.653926,...,-1.954278,0.307485,1.580450,0.0,2.261763,3.534729,2.261763,3.534729,1.272966,1


In [12]:
#dickey fuller test on each column

#regression fit and results in vertical dataframe format (column for pair id, column for dickey fuller test result)
dickey_fuller_results = {'pair' : [], 'coef' : [], 'critical_value' : []}

for column in dickey_fuller_df:
    if column == 'const':
        break
    reg = sm.OLS(endog = dickey_fuller_df[column].diff(), exog = dickey_fuller_df[['const', column]].shift(1), missing = 'drop')
    results = reg.fit()
    dickey_fuller_results['pair'].append(column)
    dickey_fuller_results['coef'].append(results.params[1])
    dickey_fuller_results['critical_value'].append(results.tvalues[1])

dickey_fuller_results_df = pd.DataFrame(dickey_fuller_results)

dickey_fuller_results_df

#compare to -3.58 from the MacKinnon tables for 1% significance level, -2.93 for 5% significance level

,pair,coef,critical_value
0,h19148725_h19381603,-0.123928,-1.040438
1,h19148725_h19679370,-0.048310,-1.055305
2,h19148725_h19835010,-0.060210,-1.211497
3,h19148725_h19925203,-0.075153,-1.765073
4,h19148725_h20303798,-0.241836,-1.940083
...,...,...,...
226,h26830868_h27377022,-0.030952,-0.618926
227,h26830868_h52106,-0.051834,-1.216498
228,h27145248_h27377022,0.192431,2.870939
229,h27145248_h52106,0.145603,1.424162


In [13]:
pair_index = dickey_fuller_results_df['critical_value'].idxmin()
pair_cv = dickey_fuller_results_df['critical_value'].min()
pair_ids = dickey_fuller_results_df['pair'].loc[pair_index]
pair_coef = dickey_fuller_results_df['coef'].loc[pair_index]

horse_a = pair_ids.split("_", 1)[0]
horse_b = pair_ids.split("_", 1)[1]
pair_df = bp_t_df[[horse_a, horse_b]]
pair_df['spread'] = pair_df[horse_a] - pair_df[horse_b]

pair_spread_sd = np.std(pair_df['spread'][0:29], ddof = 1)
pair_spread_mean = pair_df['spread'][0:29].mean()

pair_df['deviation_2sd'] = np.where(abs(pair_df['spread']) - abs(pair_spread_mean) > 2 * pair_spread_sd, True, False)
pair_df['deviation_1sd'] = np.where(abs(pair_df['spread']) - abs(pair_spread_mean) > pair_spread_sd, True, False)

print("Pair row index: " + str(pair_index), ", Pair ids: " + str(pair_ids), ", Pair DF test critical value: " + str(pair_cv), ", Pair theta: " + str(pair_coef))
print("Pair average spread: " + str(pair_spread_mean), ", Pair spread standard deviation: " + str(pair_spread_sd))

if pair_cv < - 3.58:
    print(f"{horse_a} and {horse_b} form a pair. Stationarity is not rejected at the 1% signficance level.")
elif pair_cv < - 2.93:
    print(f"{horse_a} and {horse_b} form a pair. Stationarity is not rejected at the 5% signficance level.")
else:
    print("No pair found")

Pair row index: 174 , Pair ids: h23742637_h27377022 , Pair DF test critical value: -5.570239039784303 , Pair theta: -1.069407756501654
Pair average spread: 25.002758620689658 , Pair spread standard deviation: 0.010655833990334184
h23742637 and h27377022 form a pair. Stationarity is not rejected at the 1% signficance level.


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [14]:
pair_df

,h23742637,h27377022,spread,deviation_2sd,deviation_1sd
0,50.00,25.00,25.00,False,False
1,50.00,25.00,25.00,False,False
2,50.00,25.00,25.00,False,False
3,50.00,25.00,25.00,False,False
4,50.00,25.00,25.00,False,False
5,50.00,25.00,25.00,False,False
6,50.00,25.00,25.00,False,False
7,50.00,25.00,25.00,False,False
8,50.00,25.00,25.00,False,False
9,50.00,25.00,25.00,False,False


From looking at repeated versions of the table above, on occasions where the price has drifted more than two standard deviations from the mean, it has returned to the mean after around 5 time periods (10 minutes). I will look at a pairs trading strategy where the pairs trade is made upon deviation and closed 10 minutes after the deviation.

In [15]:
k = 5 #number of periods trade is kept open
 
open_trade_df = pair_df[pair_df['deviation_2sd'] == True].loc[30:] #so that we only consider data after the first 30 periods
open_trade_idx = open_trade_df.index[0] 
    
close_trade_idx = open_trade_idx + k

#to work out what the direction of the trade should be
#the spreads are always defined as horse_a - horse_b, so if the spread > average, btl horse_a and ltb horse_b. if below average, vice versa.
if abs(pair_df['spread'].iloc[open_trade_idx]) > abs(pair_spread_mean):
    #back to lay A
    bp_a = pair_df[horse_a].iloc[open_trade_idx]
    lp_a = pair_df[horse_a].iloc[close_trade_idx]
    
    win_side_a, loss_side_a = payout(bp_a, 1, lp_a, '?')
    print(f"Win side = {win_side_a}. Loss side = {loss_side_a}.")
    
    #lay to back X
    lp_b = pair_df[horse_b].iloc[open_trade_idx]
    bp_b = pair_df[horse_b].iloc[close_trade_idx]
    
    win_side_b, loss_side_b = payout(bp_b, '?', lp_b, 1)
    print(f"Win side = {win_side_b}. Loss side = {loss_side_b}.")
    
else: 
    #lay to back Y
    lp_a = pair_df[horse_a].iloc[open_trade_idx]
    bp_a = pair_df[horse_a].iloc[close_trade_idx]
    
    win_side_a, loss_side_a = payout(bp_a, '?', lp_a, 1)
    print(f"Win side = {win_side_a}. Loss side = {loss_side_a}.")
    
    #back to lay X
    bp_b = pair_df[horse_b].iloc[open_trade_idx]
    lp_b = pair_df[horse_b].iloc[close_trade_idx]
    
    win_side_b, loss_side_b = payout(bp_b, 1, lp_b, '?')
    print(f"Win side = {win_side_b}. Loss side = {loss_side_b}.")

Win side = -0.1968774395003905. Loss side = -0.19687743950039027.
Win side = -0.031353135313530345. Loss side = -0.031353135313531455.


## TO DO:
## PAYOFFS ABOVE ARE WRONG FOR NEGATIVE SPREADS
## CHANGE EARLIER CODE TO LIST ALL PAIRS, NOT JUST ONE WITH THE MOST STATIONARY SERIES